# BWM Full Coverage (static img and video)
We'll load here all the trajectories of the insertions in the bwm as well as the positions of the clusters in every insertion. Then we'll render these inside of the Allen CCF volume renderer.

In [1]:
# urn and other imports
import unityneuro.render as urn
import numpy as np
import pandas as pd
# ibl imports
from one.api import ONE
one = ONE(base_url='https://alyx.internationalbrainlab.org')
from brainbox.io.one import SpikeSortingLoader
from ibllib.atlas import AllenAtlas
atlas = AllenAtlas(25)

C:\Users\Dan\.conda\envs\iblenv\lib\site-packages\one\api.py:1416: UserWarning: Newer cache tables require ONE version 1.13.0 or greater
  warnings.warn(f'Newer cache tables require ONE version {min_version} or greater')


# Define helper functions

In [2]:
def get_bwm_sessions():

    '''
    Gaelle, 06.04.2022
    '''

    str_query = (  
       'session__project__name__icontains,ibl_neuropixel_brainwide_01,'
       'session__json__IS_MOCK,False,session__qc__lt'
       ',50,~json__qc,CRITICAL,'
       'session__extended_qc__behavior,1,'
       'json__extended_qc__tracing_exists,True,'
       '~session__extended_qc___task_stimOn_goCue_delays__lt,0.9,'
       '~session__extended_qc___task_response_feedback_delays__lt,0.9,'
       '~session__extended_qc___task_wheel_move_before_feedback__lt,0.9,'
       '~session__extended_qc___task_wheel_freeze_during_quiescence__lt,0.9,'
       '~session__extended_qc___task_error_trial_event_sequence__lt,0.9,'
       '~session__extended_qc___task_correct_trial_event_sequence__lt,0.9,'
       '~session__extended_qc___task_reward_volumes__lt,0.9,'
       '~session__extended_qc___task_reward_volume_set__lt,0.9,'
       '~session__extended_qc___task_stimulus_move_before_goCue__lt,0.9,'
       '~session__extended_qc___task_audio_pre_trial__lt,0.9')                  
        
    str_query2 = (
       'session__project__name__icontains,ibl_neuropixel_brainwide_01,'
       'session__json__IS_MOCK,False,session__qc__lt,50,'
       '~json__qc,CRITICAL,session__extended_qc__behavior,1,'       
       'json__extended_qc__tracing_exists,True,'
       'session__extended_qc___experimenter_task,PASS')        
            
    eids = np.concatenate([one.alyx.rest('insertions', 'list', django = x)
                          for x in [str_query, str_query2]])

    eid_probe = set([x['session']+'_'+x['name'] for x in ins]) # pid via x['id']
    eids = [x.split('_') for x in eid_probe] 
 
    return eids

In [3]:
# def a function to pull mlapdv coordinates from a PID
def get_mlapdv_pid(pid, one, atlas):
    ssl = SpikeSortingLoader(pid=pid, one=one)
    spikes, clusters, channels = ssl.load_spike_sorting()
    if 'x' not in channels.keys():
        return

    clu = spikes.clusters
    st = spikes.times

    # convert clu to cha 
    cha = clusters.channels[clu]
    ucha = np.unique(cha)
    
    df = pd.DataFrame(columns =['eid','pid','clu','ml','ap','dv'])

    for i,ch in enumerate(ucha):
        x = channels['x'][ch]
        y = channels['y'][ch]
        z = channels['z'][ch]
        mlapdv = atlas.xyz2ccf([x,y,z])
        df.loc[i] = [one.pid2eid(pid), pid, i, mlapdv[0], mlapdv[1], mlapdv[2]]
        
    return df

In [27]:
# def a function to get the pid for an eid/probe combo
def eid2pids(eid):
    insertions = one.alyx.rest('insertions', 'list', session=eid)
    pids = [i['id'] for i in insertions]
    return pids

In [28]:
# convenience function for colors
import random
r = lambda: random.randint(0,255)
randHexColor = lambda : '#%02X%02X%02X' % (r(),r(),r())

# Load data and save to csv

In [7]:
ins = get_bwm_sessions()

In [8]:
ins = ins[0:2]

In [9]:
ins

[['d832d9f7-c96a-4f63-8921-516ba4a7b61f', 'probe01'],
 ['948fd27b-507b-41b3-bdf8-f9f5f0af8e0b', 'probe01']]

In [11]:
alldata = pd.DataFrame(columns =['eid','pid','clu','ml','ap','dv'])
for i, (eid,probe) in enumerate(ins):
    pids = eid2pids(eid)
    for pid in pids:
        data = get_mlapdv_pid(pids[0],one,atlas)
        alldata = pd.concat([alldata, data], axis=0)

C:\Users\Dan\Downloads\ONE\alyx.internationalbrainlab.org\wittenlab\Subjects\ibl_witten_17\2020-07-22\003\alf\probe00\p
C:\Users\Dan\Downloads\ONE\alyx.internationalbrainlab.org\wittenlab\Subjects\ibl_witten_17\2020-07-22\003\alf\probe00\p
C:\Users\Dan\Downloads\ONE\alyx.internationalbrainlab.org\wittenlab\Subjects\ibl_witten_17\2020-07-22\003\alf\probe00\p
C:\Users\Dan\Downloads\ONE\alyx.internationalbrainlab.org\wittenlab\Subjects\ibl_witten_17\2020-07-22\003\alf\probe00\p
C:\Users\Dan\Downloads\ONE\alyx.internationalbrainlab.org\wittenlab\Subjects\ibl_witten_17\2020-07-22\003\alf\probe00\p
C:\Users\Dan\Downloads\ONE\alyx.internationalbrainlab.org\wittenlab\Subjects\ibl_witten_17\2020-07-22\003\alf\probe00\p
C:\Users\Dan\Downloads\ONE\alyx.internationalbrainlab.org\wittenlab\Subjects\ibl_witten_17\2020-07-22\003\alf\probe00\p
C:\Users\Dan\Downloads\ONE\alyx.internationalbrainlab.org\wittenlab\Subjects\ibl_witten_17\2020-07-22\003\alf\probe00\p
C:\Users\Dan\Downloads\ONE\alyx.internat

In [12]:
alldata.to_csv('insertions.csv')

In [31]:
alldata = pd.read_csv('insertions.csv')

In [32]:
alldata

,Unnamed: 0,eid,pid,clu,ml,ap,dv
0,0,"('948fd27b-507b-41b3-bdf8-f9f5f0af8e0b', 'prob...",b939cc85-6028-404a-995d-28c8405a07db,0,4794.0,9792.0,4411.0
1,1,"('948fd27b-507b-41b3-bdf8-f9f5f0af8e0b', 'prob...",b939cc85-6028-404a-995d-28c8405a07db,1,4794.0,9792.0,4411.0
2,2,"('948fd27b-507b-41b3-bdf8-f9f5f0af8e0b', 'prob...",b939cc85-6028-404a-995d-28c8405a07db,2,4792.0,9790.0,4391.0
3,3,"('948fd27b-507b-41b3-bdf8-f9f5f0af8e0b', 'prob...",b939cc85-6028-404a-995d-28c8405a07db,3,4791.0,9789.0,4371.0
4,4,"('948fd27b-507b-41b3-bdf8-f9f5f0af8e0b', 'prob...",b939cc85-6028-404a-995d-28c8405a07db,4,4787.0,9785.0,4331.0
...,...,...,...,...,...,...,...
417,206,"('948fd27b-507b-41b3-bdf8-f9f5f0af8e0b', 'prob...",b939cc85-6028-404a-995d-28c8405a07db,206,4167.0,9592.0,1044.0
418,207,"('948fd27b-507b-41b3-bdf8-f9f5f0af8e0b', 'prob...",b939cc85-6028-404a-995d-28c8405a07db,207,4159.0,9590.0,1005.0
419,208,"('948fd27b-507b-41b3-bdf8-f9f5f0af8e0b', 'prob...",b939cc85-6028-404a-995d-28c8405a07db,208,4155.0,9590.0,985.0
420,209,"('948fd27b-507b-41b3-bdf8-f9f5f0af8e0b', 'prob...",b939cc85-6028-404a-995d-28c8405a07db,209,4147.0,9588.0,946.0


# Build the renderer scene

In [33]:
urn.close()

(URN) disconnected from server


In [34]:
urn.setup()

(URN) connected to server
Login sent with ID: Dan


In [36]:
urn.clear()

# urn.load_beryl_areas()
# urn.set_allen_volume_visibility(True)
# # rotate the camera to look down from the front
# urn.set_camera_rotation(45,0,135)

# # add the insertions
# neuronNames = []
# neuronPos = {}
# neuronColor = {}
# neuronShape = {}

# probeColors = {}

# i = 0
# for _, row in alldata.iterrows():
    
#     if not row.probe in probeColors.keys():
#         probeColors[row.probe] = randHexColor()
#     color = probeColors[row.probe]
#     # get a new random color
#     name = 'n'+str(i)
#     i+=1
    
#     neuronNames.append(name)
#     neuronPos[name] = [row.ml, row.ap, row.dv]
#     neuronColor[name] = hexColor
#     neuronShape[name] = 'sphere'

# urn.create_neurons(neuronNames)
# urn.set_neuron_positions(neuronPos)
# urn.set_neuron_colors(neuronColor)
# urn.set_neuron_shapes(neuronShape)

AttributeError: 'Series' object has no attribute 'probe'

(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan
(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan
(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan
(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan
(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan
(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan
(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan
(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan
(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan
(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan
